In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx to Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx
Saving esami coorti 2016-2018 - 2020 02 21 per AI.xlsx to esami coorti 2016-2018 - 2020 02 21 per AI.xlsx
User uploaded file "Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx" with length 6481955 bytes
User uploaded file "esami coorti 2016-2018 - 2020 02 21 per AI.xlsx" with length 15391182 bytes


In [ ]:
import pandas as pd
import numpy as np

#Read dataset
features =  pd.read_excel("Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx")
esami = pd.read_excel("esami coorti 2016-2018 - 2020 02 21 per AI.xlsx")

Siamo interessati al totale dei CFU ottenuti e la media dei voti ottenuti, inoltre abbiamo tenuto soltanto quelli del primo semestre

In [ ]:
esami = esami.drop('Cod.Orientamento', axis=1)
esami = esami.drop('Cod.Indirizzo', axis=1)
esami = esami.drop('Cod.Materia', axis=1)
esami = esami.drop('Materia', axis=1)
index_name = esami[(esami['Giorno_esame'].dt.month > 2) & (esami['Giorno_esame'].dt.month < 11)].index
esami.drop(index_name, inplace = True)

Calcoliamo la somma dei CFU per ogni studente la mergiamo con i dati di ogni studente e i valori NaN dei CFU verranno rimpiazzati con 0

In [ ]:
tot_cfu = esami.groupby('ID_Studente')['CFUsuperati'].sum()
features = features.merge(tot_cfu,how='outer', on='ID_Studente')
features['CFUsuperati'] = features['CFUsuperati'].fillna(0)

Calcoliamo la media di voti per ogni studente e la mergiamo con i dati di ogni studente e i valori NaN dei voti verranno rimpiazzati con 0

In [ ]:
esami = esami.drop('CFUsuperati', axis=1)
esami = esami.dropna()
media_voti = esami.groupby('ID_Studente')['Voto_se_numerico'].mean()
features = features.merge(media_voti,how='outer', on='ID_Studente')
features['Voto_se_numerico'] = features['Voto_se_numerico'].fillna(0)

Rimpiazziamo i codici relativi a determinate feature con i nomi delle feature, così da poter utilizzare la funzione get_dummies() ed ottenere un one-hot-encoding

In [ ]:
features["Diploma_scuola_superiore"] = features["Diploma_scuola_superiore"].map({1: "Classico", 2: "Scientifico", 3: "Linguistico", 4: "Magistrale", 5: "Artistico", 6: "Tecnico", 7: "Professionale", 8: "Altro_italiano", 9: "Estero", 99: "Non_disponibile"})
features["area_geografica_scuolasuperiore"] = features["area_geografica_scuolasuperiore"].map({1: "Emilia_romagna", 2: "Nord", 3: "Centro", 4: "Sud_isole", 5: "Estero", 99: "Non_disponibile"})
features["area_geografica_residenza"] = features["area_geografica_residenza"].map({1: "Emilia_romagna", 2: "Nord", 3: "Centro", 4: "Sud_isole", 5: "Estero", 99: "Non_disponibile"})
features["Ambito"] = features["Ambito"].map({1: "Economia", 2: "Farmacia", 3: "Giurisprudenza", 4: "Ingegneria", 5: "Lingue", 6: "Medicina", 7: "Veterinaria", 8: "Psicologia", 9: "Scienze", 10: "Scienze_agroalimanetari", 11: "Scienze_educazione_formazione", 12: "Scienze_motorie", 13: "Scienze_politiche", 14: "Scienze_statistiche", 15: "Sociologia", 16: "Studi_umanistici"})
features = pd.get_dummies(features)

Create and add age in DataFrame

In [ ]:
from datetime import datetime, date

def getAge(dataNascita, coorte):
    ts = pd.to_datetime(str(dataNascita)) 
    born = ts.strftime('%d/%m/%Y')
    born = datetime.strptime(born, "%d/%m/%Y").date()
    date = "31/12/"+str(coorte)
    dataAttuale = datetime.strptime(date, "%d/%m/%Y").date()
    age = dataAttuale.year - born.year - ((dataAttuale.month, dataAttuale.day) < (born.month, born.day))
    return age

#add new column with NaN
features["Eta"] = np.nan

for ind in features.index:
    studente = features.loc[ind]
    dataNascita = studente["DataNascita"]
    coorte = studente["Coorte"]
    #fill the new column with age 
    features.at[ind,'Eta'] = getAge(dataNascita, coorte)

print(features)

       ID_Studente  Coorte DataNascita  Genere  voto_scuola_superiore  \
0                1    2017  1998-02-26       1                   79.0   
1                2    2016  1997-10-06       1                   75.0   
2                3    2017  1986-08-29       1                   74.0   
3                4    2017  1998-01-23       1                   63.0   
4                5    2018  1999-07-04       2                   98.0   
...            ...     ...         ...     ...                    ...   
67275        67276    2016  1997-04-22       1                   83.0   
67276        67277    2017  1998-01-08       1                  100.0   
67277        67278    2016  1997-04-16       2                   75.0   
67278        67279    2017  1995-10-28       2                    NaN   
67279        67280    2016  1992-09-15       1                   65.0   

       Classe_ISEE  Merito_ISEE  OFA_assegnati  OFA_superati   CdS  ...  \
0                5     36332.49              0  

Abbiamo sostituito i valori NaN della feature "voto_scuola_superiore" con la media. Inoltre sono state droppate le colonne non utili ai fini dell'addestramento, ed infine eliminate le tuple contente almeno un NaN

In [ ]:
mean_value = features['voto_scuola_superiore'].mean()
features['voto_scuola_superiore'] = features['voto_scuola_superiore'].fillna(mean_value)
features = features.drop('ID_Studente', axis = 1)
features = features.drop('Classe_ISEE', axis = 1)
features = features.drop('DataNascita', axis = 1)
features = features.drop('Coorte', axis = 1)
features = features.dropna()

La seguente cella serve a prescindere dal tipo di bilanciamento 

In [ ]:
#Dividing the data into attributes and labels
X = features.drop('Abbandoni', axis=1)
y = features['Abbandoni']

Abbiamo bilanciato il dataset con approccio undersampling pareggiando il numero di dropout e non droput

In [ ]:
#BILANCIAMENTO 2
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler

#Resampling data
rus = RandomUnderSampler(sampling_strategy = "majority", random_state=42)
X_res, y_res = rus.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))

X = X_res
y = y_res

Resampled dataset shape Counter({0: 2093, 1: 2093})


In [ ]:
from sklearn.model_selection import train_test_split as tts
#Dividing data into training and test sets
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.20, random_state=42)

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier

def get_stacking():
	# define the base models
	level0 = list()
	level0.append(('rf', RandomForestClassifier(n_estimators=207, random_state=42)))
	level0.append(('svm', SVC(C=0.01, kernel='linear')))
	level0.append(('lr', LogisticRegression(max_iter=4000)))
	# define meta learner model
	level1 = LogisticRegression()
	# define the stacking ensemble
	model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
	return model

In [ ]:
# get a list of models to evaluate
def get_models():
	models = dict()
	models['lr'] = LogisticRegression(max_iter=4000)
	models['svm'] = SVC(C=0.01, kernel='linear')
	models['rf'] = RandomForestClassifier(n_estimators=207, random_state=42)
	models['stacking'] = get_stacking()
	return models

In [ ]:
from sklearn import metrics

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    tn, fp, fn, tp = metrics.confusion_matrix(test_labels, predictions).ravel()
    accuracy = metrics.accuracy_score(test_labels, predictions)
    classification_report = metrics.classification_report(test_labels, predictions)
    print("Model Performance ",model, ":")
    print("true negative: ", tn)
    print("false negative: ", fn)
    print("true positive: ", tp)
    print("false positive: ", fp)
    print("Classification report:\n", classification_report)
    print("Accuracy:", accuracy)
    print("\n\n")
    return accuracy

In [ ]:
models = get_models()
results, names = list(), list()
for name, model in models.items():
  model.fit(X_train, y_train)
  accuracy = evaluate(model, X_test, y_test)
  results.append(accuracy)
  names.append(name)
print("\n\n")
print(name, accuracy)

Model Performance  LogisticRegression(max_iter=4000) :
true negative:  368
false negative:  88
true positive:  295
false positive:  87
Classification report:
               precision    recall  f1-score   support

           0       0.81      0.81      0.81       455
           1       0.77      0.77      0.77       383

    accuracy                           0.79       838
   macro avg       0.79      0.79      0.79       838
weighted avg       0.79      0.79      0.79       838

Accuracy: 0.7911694510739857



Model Performance  SVC(C=0.01, kernel='linear') :
true negative:  378
false negative:  104
true positive:  279
false positive:  77
Classification report:
               precision    recall  f1-score   support

           0       0.78      0.83      0.81       455
           1       0.78      0.73      0.76       383

    accuracy                           0.78       838
   macro avg       0.78      0.78      0.78       838
weighted avg       0.78      0.78      0.78       838

